In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from IPython.display import Image
warnings.filterwarnings('ignore')

In [23]:
class ScratchDecesionTreeClassifierDepth():
    """
    深さ1の決定木分類器のスクラッチ実装

    Parameters
    ----------
    verbose : bool
      学習過程を出力する場合はTrue
    """

    def __init__(self, verbose=False):
        # ハイパーパラメータを属性として記録
        self.verbose = verbose

    def fit(self, X, y):
        """
        決定木分類器を学習する
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        """

        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print()
        pass

    def predict(self, X):
        """
        決定木分類器を使いラベルを推定する
        """

        pass
        return

# 問題1
def GINI(y):
    labels, counts = np.unique(y)
    gini = 1 - ((counts / counts.sum()) ** 2).sum()
    return gini

# 問題2
def IG(X, y):
    n_all = len(X)
    gini = 1
    for y_value in     y.unique():
        n = len(X[y == y_value])
        gini -= (n / n_all) ** 2
    return gini